In [1]:
from datasets import load_dataset
from transformers import ASTFeatureExtractor
import torch
import numpy as np
from datasets import Audio, ClassLabel

c:\Users\Crumbz\Desktop\4TH-YEAR-1ST-SEM\THESIS\thesis-testing\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("audiofolder", data_dir="./for-2seconds")

pretrained_model = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = ASTFeatureExtractor.from_pretrained(pretrained_model, num_mel_bins=64)


model_input_name = feature_extractor.model_input_names[0]
SAMPLING_RATE = feature_extractor.sampling_rate
num_labels = len(np.unique(dataset["train"]["label"]))


Generating train split: 13956 examples [00:01, 7089.42 examples/s]
Generating validation split: 2826 examples [00:00, 6980.18 examples/s]
Generating test split: 1088 examples [00:00, 6931.79 examples/s]


In [3]:
print('dataset:', dataset)
print('dataset train features:', dataset['train'].features)
print('dataset test features:', dataset['test'].features)
print('dataset validation features:', dataset['validation'].features)
print('model_input_name:', model_input_name)
print('SAMPLING_RATE:', SAMPLING_RATE)
print('dataset["train"][0]:', dataset['train'][0])
print('num_labels:', num_labels)
print('dataset columns:', dataset['train'].column_names)


dataset: DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 13956
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 2826
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 1088
    })
})
dataset train features: {'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None), 'label': ClassLabel(names=['fake', 'real'], id=None)}
dataset test features: {'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None), 'label': ClassLabel(names=['fake', 'real'], id=None)}
dataset validation features: {'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None), 'label': ClassLabel(names=['fake', 'real'], id=None)}
model_input_name: input_values
SAMPLING_RATE: 16000
dataset["train"][0]: {'audio': {'path': 'C:\\Users\\Crumbz\\Desktop\\4TH-YEAR-1ST-SEM\\THESIS\\thesis-testing\\for-2seconds\\training\\fake\\file10005.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.

In [4]:
# calculate values for normalization
feature_extractor.do_normalize = False

# Initialize running statistics
n = 0
mean = 0.0
M2 = 0.0  # For running variance calculation

def preprocess_audio(batch):
    wavs = [audio["array"] for audio in batch["input_values"]]
    inputs = feature_extractor(wavs, sampling_rate=SAMPLING_RATE, return_tensors="pt")
    return {model_input_name: inputs.get(model_input_name), "labels": list(batch["label"])}

dataset = dataset.rename_column("audio", "input_values")
dataset["train"].set_transform(preprocess_audio, output_all_columns=False)

# Process in batches to save memory
for batch in dataset["train"]:
    audio_input = batch[model_input_name]
    batch_size = audio_input.shape[0]

    # Calculate batch statistics
    batch_mean = torch.mean(audio_input)
    batch_variance = torch.var(audio_input, unbiased=False)  # Use N instead of N-1 for population variance
    
    # Update running statistics using Welford's online algorithm
    delta = batch_mean - mean
    mean += delta * batch_size / (n + batch_size)
    M2 += batch_variance * batch_size + delta ** 2 * n * batch_size / (n + batch_size)
    n += batch_size

# Calculate final statistics
feature_extractor.mean = mean.item()
feature_extractor.std = torch.sqrt(M2 / n).item()  # Population standard deviation
feature_extractor.do_normalize = True

print('mean: ', feature_extractor.mean)
print('std: ', feature_extractor.std)

mean:  -0.8112587332725525
std:  2.486414670944214


In [5]:
from audiomentations import Compose, AddGaussianSNR, GainTransition, Gain, ClippingDistortion, TimeStretch, PitchShift

audio_augmentations = Compose([
    AddGaussianSNR(min_snr_db=10, max_snr_db=20),
    Gain(min_gain_db=-6, max_gain_db=6),
    GainTransition(min_gain_db=-6, max_gain_db=6, min_duration=0.01, max_duration=0.3, duration_unit="fraction"),
    ClippingDistortion(min_percentile_threshold=0, max_percentile_threshold=30, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2),
    PitchShift(min_semitones=-4, max_semitones=4),
], p=0.8, shuffle=True)

In [6]:
def preprocess_audio_with_transforms(batch):
    # we apply augmentations on each waveform
    wavs = [audio_augmentations(audio["array"], sample_rate=SAMPLING_RATE) for audio in batch["input_values"]]
    inputs = feature_extractor(wavs, sampling_rate=SAMPLING_RATE, return_tensors="pt")

    output_batch = {model_input_name: inputs.get(model_input_name), "labels": list(batch["labels"])}

    return output_batch

# Cast the audio column to the appropriate feature type and rename it
dataset = dataset.cast_column("audio", Audio(sampling_rate=feature_extractor.sampling_rate))

In [7]:
# with augmentations on the training set
dataset["train"].set_transform(preprocess_audio_with_transforms, output_all_columns=False)
# w/o augmentations on the test set
dataset["test"].set_transform(preprocess_audio, output_all_columns=False)

In [8]:
temp_ds = dataset['train'].with_format(None)  # This removes any applied transforms
temp_ds_test = dataset['test'].with_format(None)  # This removes any applied transforms

print(temp_ds[0])
unique_labels = sorted(set(temp_ds["label"] + temp_ds_test["label"]))
print(unique_labels)

{'input_values': {'path': 'C:\\Users\\Crumbz\\Desktop\\4TH-YEAR-1ST-SEM\\THESIS\\thesis-testing\\for-2seconds\\training\\fake\\file10005.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', 'array': array([ 0.10552979,  0.11013794,  0.00952148, ..., -0.20837402,
       -0.25552368, -0.24255371], shape=(32000,)), 'sampling_rate': 16000}, 'label': 0, 'audio': None}
[0, 1]


In [9]:
from transformers import ASTConfig, ASTForAudioClassification

# Load configuration from the pretrained model
config = ASTConfig.from_pretrained(pretrained_model)

# Update configuration with the number of labels in our dataset
config.num_labels = num_labels
config.label2id = {"fake": 0, "real": 1}  # Maps class names to their numeric IDs
config.id2label = {0: "fake", 1: "real"}  # Maps numeric IDs back to class names

# Initialize the model with the updated configuration
model = ASTForAudioClassification.from_pretrained(pretrained_model, config=config, ignore_mismatched_sizes=True)
model.init_weights()

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
